### Intuition
Part 1 - Getting the Dataset from cornel movie dataset

Part 2 - Data Preprocessing

Part 3 - Building the Seq2Seq model

Part 4 - Training the Seq2Seq model

Part 5 - Testing the Seq2Seq model

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import time


In [4]:
lines = open(r"C:\Users\Ananta Arora\Documents\Machine Learning A-Z\Natural_Language_Processing\cornell movie-dialogs corpus\movie_lines.txt", encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open(r"C:\Users\Ananta Arora\Documents\Machine Learning A-Z\Natural_Language_Processing\cornell movie-dialogs corpus\movie_conversations.txt", encoding = 'utf-8', errors = 'ignore').read().split('\n')
lines

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?',
 'L868 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ The "real you".',
 'L867 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ What good stuff?',
 "L866 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ I figured yo

In [5]:
conversations

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

In [6]:
## Creating a dictionary that mas each line and its id 
id2line = {}
for line in  lines :
    _line =  line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
id2line

{'L637809': "...that's true.  Sometimes they change, sometimes they don't. Now, I remember back to when you were disbarred...",
 'L393826': 'Y-yes.',
 'L55323': 'Helen.  I appeal to you -',
 'L308960': 'Did your servant got word to Dr. Drossos?',
 'L221218': "That's Frances. I'm not the cookbook.",
 'L575778': 'Should I... should I tell you why?',
 'L125129': 'He did sometimes.',
 'L110968': 'Ah.  Would you mind if I borrowed your computer console?',
 'L485021': "Listen, no need for you to worry. All that stuff that happened before is behind us. Let's just try to move on.",
 'L254263': "Been out travelin' around?",
 'L113873': 'Yes.',
 'L276262': "I'm stumped.",
 'L168677': "Good Lord.  So the construction guys say it'll take about a week to fix the roof. Damn airline better not fuck us on the shingle match.",
 'L101664': '"We just got up and left. Bum musta rolled them before the police got there."',
 'L402428': "It doesn't matter, 'cause they're not regular horses, they got wings, an

#### Creating the list of all the conversations

In [7]:
conversations_ids = []
for conversation in conversations[:-1]:# except the last row as it is empty in ur dataset
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ", "")
    conversations_ids.append(_conversation.split(","))

#### Getting separately the questions and the answers

In [8]:
questions = []
answers = []
for  conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

### Doing a first cleaning of the texts


In [9]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)  
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", " ", text)
    return text

### cleaning  the questions

In [10]:
clean_questions = [] 
for question in questions:
    clean_questions.append(clean_text(question))

### Cleaning the answers

In [11]:
clean_answers = [] 
for answer in answers:
    clean_answers.append(clean_text(answer))

In [12]:
clean_questions

['can we make this quick   roxanne korrine and andrew barrett are having an incredibly horrendous public break  up on the quad   again ',
 'well  i thought we would start with pronunciation  if that is okay with you ',
 'not the hacking and gagging and spitting part   please ',
 'you are asking me out   that is so cute  what is your name again ',
 "no  no  it's my fault    we didn't have a proper introduction    ",
 'cameron ',
 'the thing is  cameron    i am at the mercy of a particularly hideous breed of loser   my sister   i cannot date until she does ',
 'why ',
 'unsolved mystery   she used to be really popular when she started high school  then it was just like she got sick of it or something ',
 'gosh  if only we could find kat a boyfriend   ',
 "c'esc ma tete  this is my head",
 'right   see   you are ready for the quiz ',
 "i don't want to know how to say that though   i want to know useful things  like where the good stores are   how much does champagne cost   stuff like chat

In [13]:
clean_answers

['well  i thought we would start with pronunciation  if that is okay with you ',
 'not the hacking and gagging and spitting part   please ',
 "okay    then how 'bout we try out some french cuisine   saturday   night ",
 'forget it ',
 'cameron ',
 'the thing is  cameron    i am at the mercy of a particularly hideous breed of loser   my sister   i cannot date until she does ',
 'seems like she could get a date easy enough   ',
 'unsolved mystery   she used to be really popular when she started high school  then it was just like she got sick of it or something ',
 'that is a shame ',
 'let me see what i can do ',
 'right   see   you are ready for the quiz ',
 "i don't want to know how to say that though   i want to know useful things  like where the good stores are   how much does champagne cost   stuff like chat   i have never in my life had to point out my head to someone ",
 "that is because it's such a nice one ",
 'forget french ',
 "well  there's someone i think might be   ",
 'whe

### Now we have to remove the non - frequent words

#### Creating a dictionary that maps each word to its number of  occurrences

In [14]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:#the words that are appearing first time in question
            word2count[word] = 1
        else:
            word2count[word] += 1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:#the words that are appearing first time in question
            word2count[word] = 1
        else:
            word2count[word] += 1

In [15]:
word2count

{'coordinated': 5,
 'squarefoot': 7,
 'stabber': 1,
 'cure!': 1,
 'defuse': 10,
 'torment': 13,
 "get'em": 2,
 'superiority': 9,
 'citizen!': 1,
 'vive': 3,
 'promising!': 1,
 'fack': 3,
 'camille': 7,
 'supertanker': 2,
 'stanislaus': 1,
 'facilities': 22,
 'luh': 13,
 'empossed': 1,
 'manolete': 3,
 'resurface': 1,
 'keeps': 293,
 'spirits': 67,
 'woodwind': 1,
 'ebola': 1,
 'eraser': 5,
 'wishful': 7,
 '1958': 2,
 'harvest': 23,
 'meeker': 5,
 'broadcast': 29,
 'ravell!!': 2,
 'gutty': 1,
 'upset!': 4,
 'pain': 396,
 'twinkie': 6,
 'rubbing': 18,
 'burial': 26,
 'singer': 76,
 'sonnet': 5,
 'pairs': 15,
 'petticoat': 4,
 'dinged': 1,
 'indisposition': 2,
 'desk!': 9,
 "'scume": 1,
 'strobes': 1,
 "brandon's": 8,
 'pipes': 35,
 'fox': 79,
 'pinocchio': 4,
 'elegance': 2,
 'baha': 2,
 'incurring': 1,
 'methylcyanoacrylate': 2,
 'poly': 4,
 'snack': 11,
 'petitioner': 3,
 "springsteen's": 2,
 'pandora': 1,
 'zeiss': 1,
 "6's": 2,
 'weakness!': 1,
 'kids&quot': 1,
 'knowlton': 1,
 'grou

### Tokenisation and filtering the non-frequent words
** Creating a dictionary that maps each word to its number of occurences**

In [16]:
threshold = 20
questionswords2int = {}
word_number = 0 
for word, count  in word2count.items():
#that in Word to count and to get them we need to adhere to that and items with parenthesis and this will get us the word.Which are the key identifiers and the number of occurrences which are the values of identifiers in the word to count dictionary.
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1
        

answerswords2int = {}
word_number = 0 
for word, count  in word2count.items():
#that in Word to count and to get them we need to adhere to that and items with parenthesis and this will get us the word.Which are the key identifiers and the number of occurrences which are the values of identifiers in the word to count dictionary.
    if count >= threshold:
        answerswords2int[word] = word_number
        word_number += 1
#It's because we're going to simply create these unique integers by starting from zero.So the first word that we find appearing more than the threshold will be associated to zero then the
#next one that we find appearing more than the threshold will be associated to one then the next one
#to two then the next one to three. Well you get the idea.We will increment the word number
        

In [17]:
questionswords2int 

{'adult': 7792,
 'ellen': 7793,
 'witch': 8701,
 'groups': 1128,
 'stacy': 1129,
 'wake': 3342,
 'nicole': 5575,
 'fear': 5576,
 'stuff': 3343,
 'freddy': 7421,
 'gets': 3344,
 'tough': 4405,
 'grab': 6132,
 'jedi': 7494,
 'greg': 6710,
 'frances': 1130,
 'cable': 7251,
 'bishop': 4413,
 "thing's": 1132,
 'metropolis': 5577,
 'bath': 8185,
 'spirits': 2,
 'mayor': 1133,
 'tower': 1134,
 'shinzon': 5578,
 'involves': 4425,
 'superior': 5947,
 'passengers': 2270,
 'liberty': 7797,
 'holiness': 6714,
 'gwen': 2271,
 'counts': 6715,
 'harvest': 3,
 'broadcast': 4,
 'grandfather': 8849,
 'bump': 3345,
 'service': 7586,
 'ratched': 2305,
 'seasons': 4406,
 'in': 4407,
 'grange': 6717,
 'pain': 5,
 'sounded': 7794,
 'gerry': 2309,
 'sicily': 6718,
 'awkward': 3346,
 'burial': 6,
 'singer': 7,
 'flame': 8155,
 'rap': 1136,
 'bonds': 3347,
 'louis': 1137,
 'chain': 7795,
 'add': 1138,
 'cigarette': 2274,
 'resurrection': 2275,
 'impressive': 7799,
 'fox': 9,
 'plead': 2276,
 'looked': 3348,
 'l

In [18]:
answerswords2int 

{'adult': 7792,
 'ellen': 7793,
 'witch': 8701,
 'groups': 1128,
 'stacy': 1129,
 'wake': 3342,
 'nicole': 5575,
 'fear': 5576,
 'stuff': 3343,
 'freddy': 7421,
 'gets': 3344,
 'tough': 4405,
 'grab': 6132,
 'jedi': 7494,
 'greg': 6710,
 'frances': 1130,
 'cable': 7251,
 'bishop': 4413,
 "thing's": 1132,
 'metropolis': 5577,
 'bath': 8185,
 'spirits': 2,
 'mayor': 1133,
 'tower': 1134,
 'shinzon': 5578,
 'involves': 4425,
 'superior': 5947,
 'passengers': 2270,
 'liberty': 7797,
 'holiness': 6714,
 'gwen': 2271,
 'counts': 6715,
 'harvest': 3,
 'broadcast': 4,
 'grandfather': 8849,
 'bump': 3345,
 'service': 7586,
 'ratched': 2305,
 'seasons': 4406,
 'in': 4407,
 'grange': 6717,
 'pain': 5,
 'sounded': 7794,
 'gerry': 2309,
 'sicily': 6718,
 'awkward': 3346,
 'burial': 6,
 'singer': 7,
 'flame': 8155,
 'rap': 1136,
 'bonds': 3347,
 'louis': 1137,
 'chain': 7795,
 'add': 1138,
 'cigarette': 2274,
 'resurrection': 2275,
 'impressive': 7799,
 'fox': 9,
 'plead': 2276,
 'looked': 3348,
 'l

#### Adding the last tokens to these two dictionaries that are :
sos and eos

In [19]:
tokens = [ '<PAD>' , '<EOS>', '<OUT>', '<SOS>' ]
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1
#Thanks to these dictionaries we know which words are not the five percent least we can word so later on in one or two tutorials.
#We will use these dictionaries to replace all the words all these five percent less frequent words.
#By this token out so that these 5 percent less frequent words are all replaced by one same common token
#which will be out.So that's why it's very important for us right now to create these tokens. 

#### Inverse Mapping

> we're going to create the inverse dictionary of the answers were in dictionary.
**Why do we need to do that? **That's because we will need inverse mapping from the integers to the answers words in the implementation of the sectors like model.And we actually just need for the answers words in dictionary and not the questions word.

In [20]:
answersint2word = {w_i: w for w , w_i in  answerswords2int.items()}
#w_i = integers of the words/values of the words
#w = key identifiers to this dictionary

In [21]:
answersint2word 

{0: 'facilities',
 1: 'keeps',
 2: 'spirits',
 3: 'harvest',
 4: 'broadcast',
 5: 'pain',
 6: 'burial',
 7: 'singer',
 8: 'pipes',
 9: 'fox',
 10: 'wade',
 11: 'dressing',
 12: 'spell',
 13: 'union',
 14: "shit's",
 15: 'fool!',
 16: 'jefferson',
 17: 'faggot',
 18: 'thus',
 19: 'carlton',
 20: 'fifth',
 21: 'of',
 22: 'month',
 23: 'so',
 24: 'forbid',
 25: 'slash',
 26: 'ordinary',
 27: 'lands',
 28: 'tony',
 29: 'stewardess',
 30: 'bought',
 31: 'questions',
 32: 'tend',
 33: 'killer',
 34: 'flare',
 35: 'furniture',
 36: 'subject',
 37: '30',
 38: 'beauty',
 39: 'read',
 40: 'coats',
 41: 'recruiting',
 42: 'leaves',
 43: 'cellar',
 44: 'something',
 45: 'radiation',
 46: 'fences',
 47: 'ship',
 48: 'good!',
 49: "sam's",
 50: 'too!',
 51: 'closer',
 52: 'prison',
 53: 'car',
 54: 'custom',
 55: 'twice',
 56: 'hafta',
 57: 'genes',
 58: 'platform',
 59: 'portion',
 60: 'corpses',
 61: "d'you",
 62: 'kurt',
 63: 'hill',
 64: 'france',
 65: 'manager',
 66: 'mattress',
 67: 'burger',


#### Adding the End of string token to the end of every answer

In [22]:
for i in range(len(clean_answers)):
    clean_answers[i] += "<EOS>"
#Eos token is needed at the end of decoding layers of the sec to sec model

In [23]:
clean_answers

['well  i thought we would start with pronunciation  if that is okay with you <EOS>',
 'not the hacking and gagging and spitting part   please <EOS>',
 "okay    then how 'bout we try out some french cuisine   saturday   night <EOS>",
 'forget it <EOS>',
 'cameron <EOS>',
 'the thing is  cameron    i am at the mercy of a particularly hideous breed of loser   my sister   i cannot date until she does <EOS>',
 'seems like she could get a date easy enough   <EOS>',
 'unsolved mystery   she used to be really popular when she started high school  then it was just like she got sick of it or something <EOS>',
 'that is a shame <EOS>',
 'let me see what i can do <EOS>',
 'right   see   you are ready for the quiz <EOS>',
 "i don't want to know how to say that though   i want to know useful things  like where the good stores are   how much does champagne cost   stuff like chat   i have never in my life had to point out my head to someone <EOS>",
 "that is because it's such a nice one <EOS>",
 'for

#### Translating all the questions and answers into integers and replacing all the words that were filtered out by OUT

Why we are doing this ?
 > we are only doing this because we want to sort all the questions and all the answers by their length to optimise the training performance.
 
 > For eg; It's like if you're training a child to speak English or French will you not teach him English with long sentences you will start with some short sentences. Otherwise the child will get overwhelmed.

In [24]:
questions_to_int = []
for questions in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append( questionswords2int[word])
    questions_to_int.append(ints)

In [25]:
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

In [26]:
answers_into_int

[[4446,
  2670,
  2020,
  7997,
  896,
  4690,
  7273,
  8869,
  8855,
  4601,
  7780,
  4304,
  7273,
  5305,
  8868],
 [3102, 6147, 8869, 514, 8869, 514, 8869, 1566, 5846, 8868],
 [4304, 2643, 6441, 8278, 7997, 1658, 3215, 883, 4208, 8869, 2479, 8491, 8868],
 [1527, 6185, 8868],
 [1326, 8868],
 [6147,
  5130,
  7780,
  1326,
  2670,
  5537,
  3083,
  6147,
  375,
  21,
  1303,
  2103,
  8869,
  5460,
  21,
  7924,
  5955,
  8791,
  2670,
  2098,
  3830,
  2631,
  446,
  7027,
  8868],
 [5808, 3110, 446, 6309, 2078, 1303, 3830, 3945, 1101, 8868],
 [8869,
  5017,
  446,
  6559,
  5236,
  3783,
  8295,
  3992,
  5881,
  446,
  1824,
  7990,
  241,
  2643,
  6185,
  7242,
  8244,
  3110,
  446,
  2604,
  6151,
  21,
  6185,
  8283,
  44,
  8868],
 [4601, 7780, 1303, 3850, 8868],
 [5720, 6483, 6367, 1067, 2670, 1795, 5670, 8868],
 [4040, 6367, 5305, 7708, 2716, 6936, 6147, 8869, 8868],
 [2670,
  5996,
  4359,
  5236,
  999,
  6441,
  5236,
  3362,
  4601,
  4214,
  2670,
  4359,
  5236,
 

In [27]:
questions_to_int

[[514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,

#### Sorting the questions and answers by the length

In [28]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1,25+1):
    # 25 + 1 because upperbound range in python are excluded so to add 25 .
# we are going to use the enumerate function to enumerate questions  into list   
     for i in enumerate(questions_to_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_to_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])
# i[1] means index of the question if i[2] then means we are cheacking the question itself
        

In [29]:
sorted_clean_answers

[[4446,
  2670,
  2020,
  7997,
  896,
  4690,
  7273,
  8869,
  8855,
  4601,
  7780,
  4304,
  7273,
  5305,
  8868],
 [3102, 6147, 8869, 514, 8869, 514, 8869, 1566, 5846, 8868],
 [4304, 2643, 6441, 8278, 7997, 1658, 3215, 883, 4208, 8869, 2479, 8491, 8868],
 [1527, 6185, 8868],
 [1326, 8868],
 [6147,
  5130,
  7780,
  1326,
  2670,
  5537,
  3083,
  6147,
  375,
  21,
  1303,
  2103,
  8869,
  5460,
  21,
  7924,
  5955,
  8791,
  2670,
  2098,
  3830,
  2631,
  446,
  7027,
  8868],
 [5808, 3110, 446, 6309, 2078, 1303, 3830, 3945, 1101, 8868],
 [8869,
  5017,
  446,
  6559,
  5236,
  3783,
  8295,
  3992,
  5881,
  446,
  1824,
  7990,
  241,
  2643,
  6185,
  7242,
  8244,
  3110,
  446,
  2604,
  6151,
  21,
  6185,
  8283,
  44,
  8868],
 [4601, 7780, 1303, 3850, 8868],
 [5720, 6483, 6367, 1067, 2670, 1795, 5670, 8868],
 [4040, 6367, 5305, 7708, 2716, 6936, 6147, 8869, 8868],
 [2670,
  5996,
  4359,
  5236,
  999,
  6441,
  5236,
  3362,
  4601,
  4214,
  2670,
  4359,
  5236,
 

In [30]:
sorted_clean_questions


[[514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,
  3859,
  177,
  7823,
  713],
 [514,
  2670,
  1475,
  5305,
  5305,
  5670,
  3102,
  4407,
  3156,
  2670,
  896,
  3783,
  5112,
  6936,
  177,
  2628,
  2781,
  1067,

 #### Tensorflow
 - They allow faster computations in deep neural networks
 - We need to create placeholder variables because it contains tensors and other inputs.

### Creating placeholders for the inputs and the targets

In [31]:
# This function will take inputs and convert them into tensorflow placeholders

def model_inputs():
    inputs = tf.placeholder(tf.int32 , [None, None], name = "inputs" )
    targets = tf.placeholder(tf.int32 , [None, None], name = "targets" )
#Learning rate = lr and there is going to be no hyper parameter
    lr = tf.placeholder(tf.float32 , name = "learning_rate" )
    keep_prob = tf.placeholder(tf.float32 , name = "keep_prob" )

    return inputs, targets,lr, keep_prob
    
    
# we took none and none here because in our inputs the questions are in form of list of matriz 2 dimensional so none and none represents 2 dimensional
# remember that the answers are the targets
# Keep_prob hyperparameter is used to control the droupout rate because it is high and low sometimes so we need to increase and decrease it.
# The dropout rate of the neurons used to override during one iteration in the training.
# usually the dropout rate is 20% so that you can activate 20% of the neurons during the different iterations of the training
    

### The decoder needs a special format for targets to decode.
- The targets must be into batches.we need to create batch size .
- Each answer in the batch of targets must start with the SOS tokens.
- The recurrent neural network of the decoder will not accept single targets/single answers.



#### Preprocessing the targets


In [32]:
def preprocess_targets(targets , word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    # 1 means 1 column
    right_side = tf.strided_slice(targets,[0,0],[batch_size,-1], [1,1])
    #strided_slice function extracts a subset of a tensor
    #-1 is to not add the the last column
    preprocessed_targets = tf.concat([left_side,right_side],1)
    return preprocessed_targets

### Architecture of RNN

#### Creating the encoder RNN layer

In [33]:
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob,sequence_length):
#sequence_length is the List of the length of the questions inside each batch
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout]* num_layers)
    _,encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                      cell_bw = encoder_cell,
                                                      sequence_length = sequence_length,
                                                      inputs = rnn_inputs,
                                                      dtype = tf.float32)
    
    return encoder_state
# Drop out is deactivating the certain percentage of the neurons during the training iterations
# The usual dropout rate is 20 % which means 20 % of the neurons are non-existing
#their weights are not updated
# Bidirectional dynmaic rnn takes your input that we are about to answerand will build 
# independent forward and backward rnn
# Imprtant- we have to make sure the input size of the forward cell and the backward cell must match.

###  Decoder RNN Layer:
- Decoding the training set
- Decoding the Validation set


In [34]:
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length,decoding_scope,output_function,keep_prob,batch_size):
    attention_states = tf.zeros([batch_size , 1, decoder_cell.output_size])
    # no of columns = 1 
    # no of elements = decoder_cell.output_size
    attention_keys , attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = 'bahdanau', num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0], attention_keys ,
                                                                              attention_values, 
                                                                              attention_score_function, 
                                                                              attention_construct_function,
                                                                              name ='attn_dec_train')
    decoder_output, _ , _ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell, 
                                                                   training_decoder_function,
                                                                   decoder_embedded_input,
                                                                   sequence_length,
                                                                   scope = decoding_scope)
                                                            
    
    
    
    
    
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)
    
                                                                              
    
#The attention keys that's the key is to be compared with the target states 
# The attention values that's the values that we'll use to construct the context vectors.
#context is returned by the encoder and that should be used by the decoder as the first element of the decoding.
# Then the attention's score function is used to compute the similarity between the keys and the target states and eventually the attention constructs function is a function used to build the tension state.
#So we have all these attention features.
#training decoder function that wil do the decoding of the training set
# Dynamic_rnn_decoder is a functionthat is used to get the output to final state
#and the final context state of the  decoder and it returns decoder_output then the decoder_final_state then the decoder_final_context_state
#but we are only inetersted in decoder_output so we will put a __

### Decoding the test/validation set

In [35]:
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, sos_id, eos_id, maximum_length, num_words, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")
    test_predictions,_, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                test_decoder_function,
                                                                scope = decoding_scope)
    # we removed dropout because we dont need it in test sets
    return test_predictions

###  Creating the Decoder RNN

In [36]:
# Creating the Decoder RNN
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix,
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
    return training_predictions, test_predictions

### Building the seq2seq model

In [37]:
# Building the seq2seq model
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix,
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    return training_predictions, test_predictions

### Training the Seq2Seq model

#### Setting the hyperparameters

In [45]:
epochs = 100
batch_size = 64
rnn_size = 512
num_layers = 3
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5
#So that's basically by which present age the learning rate is reduced over the iterations of the training because you know we want to start with a certain learning rate but then want to play decay over the
#iterations of the training to reduce the learning rate so that it can learn in more depth the logic of human conversations.
 
# Defining a session
tf.reset_default_graph()
session = tf.InteractiveSession()
 
# Loading the model inputs
inputs, targets, lr, keep_prob = model_inputs()
 
# Setting the sequence length
sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')
 
# Getting the shape of the inputs tensor
input_shape = tf.shape(inputs)

# Getting the training and test predictions
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)
 


#### # Setting up the Loss Error, the Optimizer and Gradient Clipping

In [57]:
# Setting up the Loss Error, the Optimizer and Gradient Clipping
with tf.name_scope("optimization"):
    loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions,
                                                  targets,
                                                  tf.ones([input_shape[0], sequence_length]))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss_error)
    clipped_gradients = [(tf.clip_by_value(grad_tensor, -5., 5.), grad_variable) for grad_tensor, grad_variable in gradients if grad_tensor is not None]
    optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)
 
 # Loss Error
    
    # Padding the sequences with the <PAD> token
def apply_padding(batch_of_sequences, word2int):
    max_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
    return [sequence + [word2int['<PAD>']] * (max_sequence_length - len(sequence)) for sequence in batch_of_sequences]
#sequence right now is a list of token /key identifiers
 
# Splitting the data into batches of questions and answers
def split_into_batches(questions, answers, batch_size):
    for batch_index in range(0, len(questions) // batch_size):
        start_index = batch_index * batch_size
        questions_in_batch = questions[start_index : start_index + batch_size]
        answers_in_batch = answers[start_index : start_index + batch_size]
        padded_questions_in_batch = np.array(apply_padding(questions_in_batch, questionswords2int))
        padded_answers_in_batch = np.array(apply_padding(answers_in_batch, answerswords2int))
        yield padded_questions_in_batch, padded_answers_in_batch
 

In [59]:
 padded_answers_in_batch

array([[ 753, 8869, 8869, ..., 8867, 8867, 8867],
       [6185, 6639, 3783, ..., 8867, 8867, 8867],
       [2948, 8869, 5305, ..., 8867, 8867, 8867],
       ...,
       [7581, 4806, 2243, ..., 8867, 8867, 8867],
       [8244, 1303, 1612, ..., 8867, 8867, 8867],
       [6178, 4040, 6178, ..., 8867, 8867, 8867]])

In [52]:
# Splitting the questions and answers into training and validation sets
training_validation_split = int(len(sorted_clean_questions) * 0.15)
training_questions = sorted_clean_questions[training_validation_split:]
training_answers = sorted_clean_answers[training_validation_split:]
validation_questions = sorted_clean_questions[:training_validation_split]
validation_answers = sorted_clean_answers[:training_validation_split]
 


In [58]:
# Training
batch_index_check_training_loss = 100
batch_index_check_validation_loss = ((len(training_questions)) // batch_size // 2) - 1
total_training_loss_error = 0
list_validation_loss_error = []
early_stopping_check = 0
early_stopping_stop = 1000
checkpoint = "./chatbot_weights.ckpt" # For Windows users, replace this line of code by: checkpoint = "./chatbot_weights.ckpt"
session.run(tf.global_variables_initializer())
for epoch in range(1, epochs + 1):
    for batch_index, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(training_questions, training_answers, batch_size)):
        starting_time = time.time()
        _, batch_training_loss_error = session.run([optimizer_gradient_clipping, loss_error], {inputs: padded_questions_in_batch,
                                                                                               targets: padded_answers_in_batch,
                                                                                               lr: learning_rate,
                                                                                               sequence_length: padded_answers_in_batch.shape[1],
                                                                                               keep_prob: keep_probability})
        total_training_loss_error += batch_training_loss_error
        ending_time = time.time()
        batch_time = ending_time - starting_time
        if batch_index % batch_index_check_training_loss == 0:
            print('Epoch: {:>3}/{}, Batch: {:>4}/{}, Training Loss Error: {:>6.3f}, Training Time on 100 Batches: {:d} seconds'.format(epoch,
                                                                                                                                       epochs,
                                                                                                                                       batch_index,
                                                                                                                                       len(training_questions) // batch_size,
                                                                                                                                       total_training_loss_error / batch_index_check_training_loss,
                                                                                                                                       int(batch_time * batch_index_check_training_loss)))
            total_training_loss_error = 0
        if batch_index % batch_index_check_validation_loss == 0 and batch_index > 0:
            total_validation_loss_error = 0
            starting_time = time.time()
            for batch_index_validation, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(validation_questions, validation_answers, batch_size)):
                batch_validation_loss_error = session.run(loss_error, {inputs: padded_questions_in_batch,
                                                                       targets: padded_answers_in_batch,
                                                                       lr: learning_rate,
                                                                       sequence_length: padded_answers_in_batch.shape[1],
                                                                       keep_prob: 1})
                total_validation_loss_error += batch_validation_loss_error
            ending_time = time.time()
            batch_time = ending_time - starting_time
            average_validation_loss_error = total_validation_loss_error / (len(validation_questions) / batch_size)
            print('Validation Loss Error: {:>6.3f}, Batch Validation Time: {:d} seconds'.format(average_validation_loss_error, int(batch_time)))
            
            
            
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate
            list_validation_loss_error.append(average_validation_loss_error)
            if average_validation_loss_error <= min(list_validation_loss_error):
                print('I speak better now!!')
                early_stopping_check = 0
                saver = tf.train.Saver()
                saver.save(session, checkpoint)
            else:
                print("Sorry I do not speak better, I need to practice more.")
                early_stopping_check += 1
                if early_stopping_check == early_stopping_stop:
                    break
        if early_stopping_check == early_stopping_stop:
                print("My apologies, I cannot speak better anymore. This is the best I can do.")
        break
print("Game Over")

Epoch:   1/100, Batch:    0/2943, Training Loss Error:  0.091, Training Time on 100 Batches: 1593 seconds
Epoch:   2/100, Batch:    0/2943, Training Loss Error:  0.188, Training Time on 100 Batches: 8146 seconds
Epoch:   3/100, Batch:    0/2943, Training Loss Error:  0.224, Training Time on 100 Batches: 2024 seconds
Epoch:   4/100, Batch:    0/2943, Training Loss Error:  0.063, Training Time on 100 Batches: 1275 seconds
Epoch:   5/100, Batch:    0/2943, Training Loss Error:  0.055, Training Time on 100 Batches: 1271 seconds
Epoch:   6/100, Batch:    0/2943, Training Loss Error:  0.032, Training Time on 100 Batches: 1247 seconds
Epoch:   7/100, Batch:    0/2943, Training Loss Error:  0.034, Training Time on 100 Batches: 1259 seconds
Epoch:   8/100, Batch:    0/2943, Training Loss Error:  0.030, Training Time on 100 Batches: 1387 seconds
Epoch:   9/100, Batch:    0/2943, Training Loss Error:  0.030, Training Time on 100 Batches: 1687 seconds
Epoch:  10/100, Batch:    0/2943, Training Los

Epoch:  79/100, Batch:    0/2943, Training Loss Error:  0.015, Training Time on 100 Batches: 2867 seconds
Epoch:  80/100, Batch:    0/2943, Training Loss Error:  0.016, Training Time on 100 Batches: 3313 seconds
Epoch:  81/100, Batch:    0/2943, Training Loss Error:  0.015, Training Time on 100 Batches: 2958 seconds
Epoch:  82/100, Batch:    0/2943, Training Loss Error:  0.015, Training Time on 100 Batches: 2906 seconds
Epoch:  83/100, Batch:    0/2943, Training Loss Error:  0.015, Training Time on 100 Batches: 2819 seconds
Epoch:  84/100, Batch:    0/2943, Training Loss Error:  0.015, Training Time on 100 Batches: 3387 seconds
Epoch:  85/100, Batch:    0/2943, Training Loss Error:  0.015, Training Time on 100 Batches: 3000 seconds
Epoch:  86/100, Batch:    0/2943, Training Loss Error:  0.014, Training Time on 100 Batches: 2942 seconds
Epoch:  87/100, Batch:    0/2943, Training Loss Error:  0.014, Training Time on 100 Batches: 2943 seconds
Epoch:  88/100, Batch:    0/2943, Training Los

In [48]:
tf.__version__


'1.0.0'

### Testing the Seq2Seq model

In [63]:
# Loading the weights and Running the session
checkpoint =  "./chatbot_weights.ckpt"
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, checkpoint)
 


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ./chatbot_weights.ckpt
	 [[Node: save_2/RestoreV2_55 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_2/Const_0, save_2/RestoreV2_55/tensor_names, save_2/RestoreV2_55/shape_and_slices)]]

Caused by op 'save_2/RestoreV2_55', defined at:
  File "D:\Anaconda3\envs\chatbot\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda3\envs\chatbot\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda3\envs\chatbot\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Anaconda3\envs\chatbot\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Anaconda3\envs\chatbot\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "D:\Anaconda3\envs\chatbot\lib\s

In [62]:
# Converting the questions from strings to lists of encoding integers
def convert_string2int(question, word2int):
    question = clean_text(question)
    return [word2int.get(word, word2int['<OUT>']) for word in question.split()]


In [ ]:
# Setting up the chat
while(True):
    question = input("You: ")
    if question == 'Goodbye':
        break
    question = convert_string2int(question, questionswords2int)
    question = question + [questionswords2int['<PAD>']] * (25 - len(question))
    fake_batch = np.zeros((batch_size, 25))
    fake_batch[0] = question
    predicted_answer = session.run(test_predictions, {inputs: fake_batch, keep_prob: 0.5})[0]
    answer = ''
    for i in np.argmax(predicted_answer, 1):
        if  answersint2word[i] == 'i':
            token = ' I'
        elif answersint2word[i] == '<EOS>':
            token = '.'
        elif answersint2word[i] == '<OUT>':
            token = 'out'
        else:
            token = ' ' + answersint2word[i]
        answer += token
        if token == '.':
            break
    print('ChatBot: ' + answer)

You: hi
ChatBot:  value bear polish corky interpol interpol interpol interpol interpol interpol interpol interpol interpol interpol ! ! suite drama ! ! ! drama chaos purely chaos
You: ok
ChatBot:  tracking tracking bear earrings flush interpol interpol interpol interpol interpol interpol interpol chaos chaos chaos chaos chaos chaos chaos chaos chaos chaos describe describe chaos
You: what the fuck are you saying?
ChatBot:  tracking tracking tracking suspicion suspicion sayer sayer virgins describe vacant bridge bridge bridge bridge grounds grounds grounds grounds grounds grounds grounds darlin' swedish purely psychiatrist
